# 🎙️ ValeTTS - Treinamento VITS2 no Google Colab Pro

## ⚠️ **INSTRUÇÕES IMPORTANTES:**
1. **Execute as células em ordem**
2. **Configure sua API key do OpenRouter** na célula 4
3. **Monitore pelo TensorBoard e relatórios LLM**
4. **Logs salvos automaticamente no Google Drive**

### 🎯 **Recursos Ativados:**
- 🤖 **Monitoramento LLM via OpenRouter** (relatórios automáticos)
- 📊 **TensorBoard** (métricas em tempo real)
- 💾 **Checkpoints automáticos** (Google Drive)
- 🎵 **Amostras de áudio** (a cada 10 épocas)

### 🔧 **Configuração otimizada para:**
- Tesla T4 (15GB VRAM) - Batch 6
- Tesla P100 (16GB VRAM) - Batch 8
- Tesla V100 (16GB VRAM) - Batch 10
- Mixed precision (FP16)
- Gradient accumulation 4x

In [ ]:
# 1. Verificar GPU e conectar Google Drive
!nvidia-smi
print("\n" + "="*60)

from google.colab import drive
import os
import torch
import psutil

# Conectar Google Drive
drive.mount('/content/drive')

# Verificar recursos
if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    # Configurar batch size baseado na GPU
    if 'T4' in device:
        recommended_batch = 6
    elif 'P100' in device:
        recommended_batch = 8
    elif 'V100' in device or 'A100' in device:
        recommended_batch = 10
    else:
        recommended_batch = 8
else:
    device = "CPU only"
    vram = 0
    recommended_batch = 2

ram = psutil.virtual_memory().total / 1e9
disk = psutil.disk_usage('/').free / 1e9

print(f"🔥 GPU: {device}")
print(f"💾 VRAM: {vram:.1f} GB")
print(f"🧠 RAM: {ram:.1f} GB")
print(f"💽 Disk Free: {disk:.1f} GB")
print(f"📊 Batch recomendado: {recommended_batch}")
print(f"✅ Google Drive conectado")

# Salvar configuração automática
GPU_CONFIG = {
    'device': device,
    'vram': vram,
    'recommended_batch': recommended_batch
}

In [ ]:
# 2. Clonar código e instalar dependências
import os
os.chdir('/content')

# Remover diretório se já existir
if os.path.exists('/content/ValeTTS-Colab'):
    !rm -rf /content/ValeTTS-Colab

# Clonar repositório
print("📦 Clonando código do GitHub...")
!git clone https://github.com/wallaceblaia/ValeTTS-Colab.git
os.chdir('/content/ValeTTS-Colab')

print("🔍 Estrutura do projeto:")
!ls -la

# Instalar dependências
print("\n📚 Instalando dependências...")
%pip install -r requirements.txt --quiet
%pip install tensorboard --quiet
%pip install openai --quiet  # Para OpenRouter

# Otimizar Tensor Cores
import torch
torch.set_float32_matmul_precision('medium')

print("\n✅ Dependências instaladas")
print("✅ Tensor Cores otimizados")
print("✅ TensorBoard pronto")
print("✅ OpenRouter configurado")

# Verificar versões
import lightning as L
import librosa
print(f"\n🔧 PyTorch: {torch.__version__}")
print(f"🔧 Lightning: {L.__version__}")
print(f"🔧 Librosa: {librosa.__version__}")

In [ ]:
# 3. Extrair dataset do Google Drive
import tarfile
import time
from pathlib import Path

# Verificar dataset
drive_dataset = "/content/drive/MyDrive/ValeTTS/Dataset-Unificado.tar.gz"
print(f"🔍 Procurando dataset: {drive_dataset}")

if not os.path.exists(drive_dataset):
    print("❌ Dataset não encontrado!")
    print("\n📋 Instruções para upload:")
    print("   1. Acesse: https://drive.google.com")
    print("   2. Crie pasta: ValeTTS")
    print("   3. Faça upload: Dataset-Unificado.tar.gz")
    print("   4. Caminho final: /content/drive/MyDrive/ValeTTS/Dataset-Unificado.tar.gz")
    raise FileNotFoundError("Dataset não encontrado no Google Drive")

# Verificar tamanho
file_size = os.path.getsize(drive_dataset) / 1e9
print(f"📁 Tamanho do arquivo: {file_size:.1f} GB")

# Extrair dataset
local_path = "/content/ValeTTS-Colab/data/generated"
Path(local_path).mkdir(parents=True, exist_ok=True)

print("📦 Extraindo dataset... (pode levar alguns minutos)")
start_time = time.time()

with tarfile.open(drive_dataset, 'r:gz') as tar:
    tar.extractall(local_path)

extract_time = time.time() - start_time
print(f"⏱️  Extração concluída em {extract_time:.1f}s")

# Verificar extração
dataset_path = f"{local_path}/Dataset-Unificado"
if os.path.exists(dataset_path):
    audio_files = list(Path(f"{dataset_path}/audio/raw").glob("*.wav"))
    metadata_size = os.path.getsize(f"{dataset_path}/metadata.csv") / 1e6
    
    print(f"✅ Dataset extraído para: {dataset_path}")
    print(f"🎵 Arquivos de áudio: {len(audio_files):,}")
    print(f"📋 Metadata: {metadata_size:.1f} MB")
    
    # Verificar alguns arquivos
    sample_files = audio_files[:3]
    print(f"\n🔍 Amostras de arquivos:")
    for f in sample_files:
        size = f.stat().st_size / 1024
        print(f"   - {f.name} ({size:.1f} KB)")
else:
    print("❌ Erro na extração do dataset!")
    raise Exception("Falha na extração")

In [ ]:
# 4. ⚠️ CONFIGURE SUA API KEY DO OPENROUTER AQUI ⚠️

# ============================================
# CONFIGURAÇÃO DO OPENROUTER
# ============================================
OPENROUTER_API_KEY = "sk-or-v1-your-api-key-here"  # ← ALTERE AQUI

# Modelo a usar (recomendado: claude ou gpt-4)
LLM_MODEL = "anthropic/claude-3.5-sonnet"  # ou "openai/gpt-4o"

# OU desabilite o monitoramento LLM:
ENABLE_LLM_MONITORING = True  # ← Mude para False se não quiser LLM
ENABLE_TENSORBOARD = True     # ← TensorBoard sempre recomendado
ENABLE_REPORTS = True         # ← Relatórios em markdown
# ============================================

print("🔧 Configuração de Monitoramento:")
print(f"   🤖 LLM Monitoring: {'✅ Ativo' if ENABLE_LLM_MONITORING else '❌ Desabilitado'}")
print(f"   📊 TensorBoard: {'✅ Ativo' if ENABLE_TENSORBOARD else '❌ Desabilitado'}")
print(f"   📋 Relatórios: {'✅ Ativo' if ENABLE_REPORTS else '❌ Desabilitado'}")

if ENABLE_LLM_MONITORING:
    if OPENROUTER_API_KEY == "sk-or-v1-your-api-key-here":
        print("\n⚠️  ATENÇÃO: Configure sua API key do OpenRouter acima!")
        print("   1. Acesse: https://openrouter.ai/keys")
        print("   2. Crie uma API key")
        print("   3. Cole acima em OPENROUTER_API_KEY")
        print("   4. Ou mude ENABLE_LLM_MONITORING para False")
    else:
        print(f"   🔑 API Key configurada: {OPENROUTER_API_KEY[:20]}...")
        print(f"   🧠 Modelo LLM: {LLM_MODEL}")
        print("   ✅ Monitoramento LLM pronto!")

# Testar conectividade se API key configurada
if ENABLE_LLM_MONITORING and OPENROUTER_API_KEY != "sk-or-v1-your-api-key-here":
    try:
        import requests
        response = requests.get(
            "https://openrouter.ai/api/v1/models",
            headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}"},
            timeout=10
        )
        if response.status_code == 200:
            print("   🌐 Conectividade OpenRouter: ✅ OK")
        else:
            print(f"   🌐 Conectividade OpenRouter: ⚠️  Status {response.status_code}")
    except Exception as e:
        print(f"   🌐 Conectividade OpenRouter: ❌ Erro: {e}")

print("\n✅ Configuração de monitoramento finalizada!")

In [ ]:
# 5. Criar configuração otimizada para Colab
import yaml
from pathlib import Path

# Configuração otimizada baseada na GPU detectada
batch_size = GPU_CONFIG['recommended_batch']

config = {
    'model': {
        'name': 'VITS2',
        'text_encoder': {
            'hidden_channels': 128,  # Otimizado para Colab
            'filter_channels': 512,  # Reduzido para economia de VRAM
            'n_heads': 2,
            'n_layers': 4,           # Reduzido de 6
            'kernel_size': 3,
            'p_dropout': 0.1
        },
        'generator': {
            'inter_channels': 128,
            'hidden_channels': 128,
            'filter_channels': 512,
            'n_heads': 2,
            'n_layers': 4,
            'resblock': '1',
            'resblock_kernel_sizes': [3, 7, 11],
            'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
            'upsample_rates': [8, 8, 2, 2],
            'upsample_initial_channel': 512,
            'upsample_kernel_sizes': [16, 16, 4, 4]
        },
        'discriminator': {
            'periods': [2, 3, 5, 7, 11],
            'period_discriminator_params': {
                'channels': 32,        # Reduzido para economia
                'max_downsample_channels': 512
            }
        }
    },
    'data': {
        'dataset_path': 'data/generated/Dataset-Unificado',
        'batch_size': batch_size,
        'sample_rate': 22050,
        'hop_length': 256,
        'win_length': 1024,
        'n_mel': 80,
        'n_fft': 1024,
        'segment_size': 8192,      # Otimizado para Colab
        'num_workers': 2           # Limitado no Colab
    },
    'training': {
        'learning_rate': 2e-4,
        'adam_b1': 0.8,
        'adam_b2': 0.99,
        'lr_decay': 0.999875,
        'batch_size': batch_size,
        'fp16_run': True,          # Mixed precision essencial
        'max_epochs': 200,
        'gradient_accumulation_steps': 4,  # Compensar batch menor
        'gradient_clip_val': 1.0,
        'warmup_epochs': 10,
        'save_every_epoch': 10,
        'validate_every_n_epoch': 5
    },
    'logging': {
        'checkpoint_dir': '/content/drive/MyDrive/ValeTTS/checkpoints',
        'log_dir': '/content/drive/MyDrive/ValeTTS/logs',
        'sample_dir': '/content/drive/MyDrive/ValeTTS/samples',
        'report_dir': '/content/drive/MyDrive/ValeTTS/reports',
        'tensorboard_enabled': ENABLE_TENSORBOARD
    },
    'llm_monitoring': {
        'enabled': ENABLE_LLM_MONITORING,
        'provider': 'openrouter',
        'api_key': OPENROUTER_API_KEY if ENABLE_LLM_MONITORING else '',
        'model': LLM_MODEL,
        'base_url': 'https://openrouter.ai/api/v1',
        'monitor_every_n_epochs': 5,
        'save_reports': ENABLE_REPORTS,
        'max_tokens': 2000,
        'temperature': 0.3
    },
    'sample_generation': {
        'enabled': True,
        'every_n_epochs': 10,
        'num_samples': 3,
        'sample_texts': [
            'Olá, este é um teste do modelo ValeTTS.',
            'A síntese de voz em português brasileiro funciona perfeitamente.',
            'Obrigado por usar o ValeTTS no Google Colab.'
        ]
    }
}

# Criar todos os diretórios necessários
directories = [
    '/content/drive/MyDrive/ValeTTS/checkpoints',
    '/content/drive/MyDrive/ValeTTS/logs',
    '/content/drive/MyDrive/ValeTTS/samples', 
    '/content/drive/MyDrive/ValeTTS/reports'
]

for dir_path in directories:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

# Salvar configuração
config_path = Path('configs/training/vits2_colab_openrouter.yaml')
config_path.parent.mkdir(parents=True, exist_ok=True)

with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

print("✅ Configuração criada e salva!")
print(f"📁 Arquivo: {config_path}")
print(f"🔥 GPU detectada: {GPU_CONFIG['device']}")
print(f"📊 Batch size: {batch_size}")
print(f"🧠 Gradient accumulation: {config['training']['gradient_accumulation_steps']}")
print(f"💾 Checkpoints: a cada {config['training']['save_every_epoch']} épocas")
print(f"🎵 Amostras: a cada {config['sample_generation']['every_n_epochs']} épocas")
print(f"🤖 LLM: {'Ativo' if ENABLE_LLM_MONITORING else 'Desabilitado'}")
print(f"📊 TensorBoard: {'Ativo' if ENABLE_TENSORBOARD else 'Desabilitado'}")
print("\n🎯 Configuração otimizada para Google Colab Pro!")

In [ ]:
# 6. 🚀 INICIAR TREINAMENTO COM MONITORAMENTO COMPLETO
from datetime import datetime
import subprocess
import threading

print("🎯 INICIANDO TREINAMENTO VITS2 NO GOOGLE COLAB PRO!")
print("="*70)
print(f"⏰ Início: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔥 GPU: {GPU_CONFIG['device']} ({GPU_CONFIG['vram']:.1f}GB VRAM)")
print(f"📊 Batch size: {batch_size} (gradient acc: 4x)")
print(f"🧠 Modelo: ~354M parâmetros")
print(f"📈 Dataset: 22,910 amostras de áudio")
print(f"🎙️ Falantes: 52 vozes diferentes")
print("")
print("🔍 MONITORAMENTO ATIVO:")
if ENABLE_LLM_MONITORING:
    print(f"   🤖 LLM Reports: {LLM_MODEL} via OpenRouter")
    print(f"       📋 Relatórios a cada 5 épocas")
if ENABLE_TENSORBOARD:
    print(f"   📊 TensorBoard: http://localhost:6006")
    print(f"       📈 Métricas em tempo real")
print(f"   💾 Checkpoints: Google Drive (a cada 10 épocas)")
print(f"   🎵 Amostras: Google Drive (a cada 10 épocas)")
print("="*70)

# Iniciar TensorBoard em background se habilitado
if ENABLE_TENSORBOARD:
    print("🚀 Iniciando TensorBoard...")
    tensorboard_process = subprocess.Popen([
        'tensorboard', '--logdir', '/content/drive/MyDrive/ValeTTS/logs',
        '--host', '0.0.0.0', '--port', '6006'
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    # Aguardar TensorBoard inicializar
    import time
    time.sleep(3)
    print("📊 TensorBoard iniciado! Acesse: http://localhost:6006")

print("\n🎯 EXECUTANDO TREINAMENTO...")
print("⏳ Este processo levará várias horas")
print("📱 Monitore pelo TensorBoard e relatórios no Google Drive")
print("\n" + "-"*50)

# EXECUTAR TREINAMENTO
!python scripts/train_vits2.py --config configs/training/vits2_colab_openrouter.yaml

print("\n" + "="*70)
print(f"⏰ Finalizado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🎉 Treinamento concluído!")
print("📁 Verifique os resultados no Google Drive: /ValeTTS/")